In [ ]:
!pip install cx_Oracle

In [18]:
import cx_Oracle
import os
import pandas as pd

os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
cursor = conn.cursor()
cursor.execute(f"""
        select cage_name from tbl_cage where cage_seq = 1 and mb_id = 'dodo'
    """)
data = cursor.fetchall()
if data != [] :
    data = {'cage_seq': 1, 'cage_name' : data[0][0]}
    cursor.execute(f"""
        select t1.kit_name, t2.sv_value
        from (select t3.kit_seq, max(t3.sv_date) as sv_date, t4.kit_name 
        from tbl_kit_sensoring t3, tbl_kit t4 
        where t4.kit_seq=t3.kit_seq and t4.cage_seq = 1
        group by t3.kit_seq, t4.kit_name) t1
        inner join tbl_kit_sensoring t2 
        on t1.kit_seq = t2.kit_seq
        where t1.sv_date = t2.sv_date
    """)
    data1 = cursor.fetchall()
    data['kit'] = dict(data1)
    try :
        temp = data['kit']['DHT11_TEMP']
        if temp <= 20 :
            data['kit']['DHT11_TEMP'] = [data['kit']['DHT11_TEMP'],'저온']
        elif temp < 30 :
            data['kit']['DHT11_TEMP'] = [data['kit']['DHT11_TEMP'],'정상']
        else :
            data['kit']['DHT11_TEMP'] = [data['kit']['DHT11_TEMP'],'고온']
        if data['kit']['DHT11_TEMP'][-1] != '정상' :
            cursor.execute(f"""
                select (SYSDATE - t1.sv_date) AS TEMP_DELAY
                from tbl_kit_sensoring t1
                inner join tbl_kit t2
                on t1.kit_seq=t2.kit_seq
                where t2.cage_seq = 1 and t2.kit_name = 'DHT11_TEMP'
                and t1.sv_date = (select MAX(sv_date) from tbl_kit_sensoring where kit_seq = t1.kit_seq AND SV_VALUE > 20 AND SV_VALUE < 30)

                union all

                select (sysdate - kit_inst_date) as TEMP_DELAY from tbl_kit where cage_seq = 1 and kit_name = 'DHT11_TEMP'
                and not exists
                (select (SYSDATE - t1.sv_date) AS TEMP_DELAY
                from tbl_kit_sensoring t1
                inner join tbl_kit t2
                on t1.kit_seq=t2.kit_seq
                where t2.cage_seq = 1 and t2.kit_name = 'DHT11_TEMP'
                and t1.sv_date = (select MAX(sv_date) from tbl_kit_sensoring where kit_seq = t1.kit_seq AND SV_VALUE > 20 AND SV_VALUE < 30))
            """)
            data0 = cursor.fetchall()
            time = float(data0[0][0])
            delay = ''
            if round(time) != 0 :
                delay += str(round(time))+'일'
            time *= 24
            if round(time) % 24 != 0 :
                delay += str(round(time) % 24)+'시간'
            time *= 60
            if round(time) % 60 != 0 :
                delay += str(round(time) % 60)+'분'
            time *= 60
            if round(time) % 60 != 0 :
                delay += str(round(time) % 60)+'초'
            data['kit']['DHT11_TEMP'] += [delay]
    except :
        pass
    try :
        humi = data['kit']['DHT11_HUMI']
        if humi < 50:
            data['kit']['DHT11_HUMI'] = [data['kit']['DHT11_HUMI'],'건조']
        elif humi < 80 :
            data['kit']['DHT11_HUMI'] = [data['kit']['DHT11_HUMI'],'정상']
        else :
            data['kit']['DHT11_HUMI'] = [data['kit']['DHT11_HUMI'],'과습']
        if data['kit']['DHT11_HUMI'][-1] != '정상' :
            cursor.execute(f"""
                select (SYSDATE - t1.sv_date) AS HUMI_DELAY2
                from tbl_kit_sensoring t1
                inner join tbl_kit t2
                on t1.kit_seq=t2.kit_seq
                where t2.cage_seq = 1 and t2.kit_name = 'DHT11_HUMI'
                and t1.sv_date = (select MAX(sv_date) from tbl_kit_sensoring where kit_seq = t1.kit_seq AND SV_VALUE > 50 AND SV_VALUE < 80)

                union all

                select (sysdate - kit_inst_date) as HUMI_DELAY from tbl_kit where cage_seq = 1 and kit_name = 'DHT11_HUMI'
                and not exists
                (select (SYSDATE - t1.sv_date) AS HUMI_DELAY2
                from tbl_kit_sensoring t1
                inner join tbl_kit t2
                on t1.kit_seq=t2.kit_seq
                where t2.cage_seq = 1 and t2.kit_name = 'DHT11_HUMI'
                and t1.sv_date = (select MAX(sv_date) from tbl_kit_sensoring where kit_seq = t1.kit_seq AND SV_VALUE > 50 AND SV_VALUE < 80))
            """)
            data0 = cursor.fetchall()
            time = float(data0[0][0])
            delay = ''
            if round(time) != 0 :
                delay += str(round(time))+'일'
            time *= 24
            if round(time) % 24 != 0 :
                delay += str(round(time) % 24)+'시간'
            time *= 60
            if round(time) % 60 != 0 :
                delay += str(round(time) % 60)+'분'
            time *= 60
            if round(time) % 60 != 0 :
                delay += str(round(time) % 60)+'초'
            data['kit']['DHT11_HUMI'] += [delay]
    except :
        pass
    cursor.execute(f"""
        select distinct t3.pet_type, listagg(t3.pet_info, '\t') WITHIN GROUP (ORDER BY t3.pet_info) AS PET_INFO
        FROM (select PET_TYPE, (PET_NAME ||'('||
        (CASE
        WHEN PET_GENDER = 'f' THEN '♀'
        WHEN PET_GENDER = 'm' THEN '♂'
        ELSE '?'
        END)
        ||')') as pet_info
        from tbl_pet WHERE CAGE_SEQ=2) T3 GROUP BY T3.PET_TYPE
        """)
    data2 = cursor.fetchall()
    data['pet'] = data2
    print(data)
cursor.close()
conn.close()

if 'HUMIDIFIER' in data['kit'].keys() :
    print(1)


{'cage_seq': 1, 'cage_name': '유토피아2', 'kit': {'DHT11_TEMP': [31.0, '고온', '2일7시간56분42초'], 'DHT11_HUMI': [28.0, '건조', '2일7시간1분36초'], 'HEATING_PAD': 1.0, 'PAD_TEMP': 40.0, 'HUMIDIFIER': 1.0, 'COOLING_PAN8*8': 1.0, 'COOLING_PAN12*12': 1.0}, 'pet': [('달팽이', '이쁜이4(?)\t이쁜이5(?)'), ('도마뱀', '이쁜이6(?)')]}
1


In [ ]:
# import cx_Oracle
# import pandas as pd
# host = 'mydbinstance.cycgpc41xdyi.us-west-1.rds.amazonaws.com'
# dsn =cx_Oracle.makedsn(host,1521,'ORCL')
# db = cx_Oracle.connect('ADMIN', "mjkAWS^930",dsn)

In [ ]:
# cursor = db.cursor()
# cursor.execute(f"""select count(mno) from member where mid = '{mid}' and mlogin='{mlogin}' """)
# row = cursor.fetchall()
# emp = pd.DataFrame(row)
# print(emp)

In [ ]:
# from flask import Flask, session, render_template, redirect, request, url_for
# from flaskext.mysql import MySQL
 
# mysql = MySQL()
# app = Flask(__name__)
 
# app.config['MYSQL_DATABASE_USER'] = 'root'
# app.config['MYSQL_DATABASE_PASSWORD'] = '0000'
# app.config['MYSQL_DATABASE_DB'] = 'user_info'
# app.config['MYSQL_DATABASE_HOST'] = 'localhost'
# app.secret_key = "ABCDEFG"
# mysql.init_app(app)
 
# @app.route('/', methods=['GET', 'POST'])
# def main():
#     error = None
 
#     if request.method == 'POST':
#         id = request.form['id']
#         pw = request.form['pw']
 
#         conn = mysql.connect()
#         cursor = conn.cursor()
#         sql = "SELECT id FROM users WHERE id = %s AND pw = %s"
#         value = (id, pw)
#         cursor.execute("set names utf8")
#         cursor.execute(sql, value)
 
#         data = cursor.fetchall()
#         cursor.close()
#         conn.close()
 
#         for row in data:
#             data = row[0]
 
#         if data:
#             session['login_user'] = id
#             return redirect(url_for('home'))
#         else:
#             error = 'invalid input data detected !'
#     return render_template('main.html', error = error)
 
 
# @app.route('/register.html', methods=['GET', 'POST'])
# def register():
#     error = None
#     if request.method == 'POST':
#         id = request.form['regi_id']
#         pw = request.form['regi_pw']
 
#         conn = mysql.connect()
#         cursor = conn.cursor()
 
#         sql = "INSERT INTO users VALUES ('%s', '%s')" % (id, pw)
#         cursor.execute(sql)
 
#         data = cursor.fetchall()
 
#         if not data:
#             conn.commit()
#             return redirect(url_for('main'))
#         else:
#             conn.rollback()
#             return "Register Failed"
 
#         cursor.close()
#         conn.close()
#     return render_template('register.html', error=error)
 
# @app.route('/home.html', methods=['GET', 'POST'])
# def home():
#     error = None
#     id = session['login_user']
#     return render_template('home.html', error=error, name=id)
 
# if __name__ == '__main__':
#     app.run()


In [ ]:
# import cx_Oracle

# #한글 지원 방법
# import os
# os.putenv('NLS_LANG', '.UTF8')

# #연결에 필요한 기본 정보 (유저, 비밀번호, 데이터베이스 서버 주소)
# connection = cx_Oracle.connect('hr','1234','localhost:1521/xe')

# cursor = connection.cursor()

# cursor.execute("""
#    select name
#    from test_db
#    where text = :texting""",
#    texting = "테스트"
# )

In [ ]:
import cx_Oracle
import os
os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
conn = cx_Oracle.connect('hr','1234','localhost:1521/xe')
cursor = conn.cursor()
cursor.execute("SELECT mid, mlogin, manager FROM member where mlogin='diamond' and mid='dodo' and mpw='1234'")
data = cursor.fetchall()
cursor.close()
conn.close()
print(data)

In [ ]:
import cx_Oracle
import os
def sql(command) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('hr','1234','localhost:1521/xe')
    cursor = conn.cursor()
    cursor.execute(command)
    data = cursor.fetchall()
    cursor.close()
    conn.close()

In [ ]:
import cx_Oracle
import os

def login_ck(mlogin,mid,mpw='') :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    if mlogin == 'diamond' :
        cursor.execute(f"SELECT mid, mlogin, manager FROM member where mlogin='diamond' and mid='{mid}'and mpw='{mpw}'")     
    else :
        cursor.execute(f"SELECT mid, mlogin, manager FROM member where mlogin='{mlogin}' and mid='{mid}'")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    
    if len(data):
        data = data[0]
        session['user_id'] =  data[0]
        session['login_by'] = data[1]
        session['manager'] = data[2]
        print('로그인 성공')
        return render_template('/home/001777')
    else :
        print('로그인 실패')
        return render_template('/login/001666')

def select_all(mlogin,mid) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"SELECT mname, mid, mtel, mmail, mlogin, manager FROM member where mlogin='{mlogin}' and mid='{mid}'")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data:
        data = data[0]
        return data
    else :
        return render_template('/404')

def register():
    
    mno = f'm{int(cursor.execute("select count(*) from member").fetchall()[0]) + 1:0<4}'
    cursor.execute(f"insert into member values('m0002', '도도', 'dodo', '1234','01012341234','dodo@gmail.com',0,'22/08/15','22/08/15');")  

def my_cage(mb_id) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""
            SELECT T1.CAGE_SEQ, T2.CAGE_NAME, T1.PET_TYPE, LISTAGG(T1.PET_NAME,',') WITHIN GROUP (ORDER BY T1.PET_NAME) AS PET
            FROM TBL_PET T1, TBL_CAGE T2
            WHERE T1.CAGE_SEQ = T2.CAGE_SEQ and T2.MB_ID = 'dodo'
            GROUP BY T1.PET_TYPE, T1.CAGE_SEQ, T2.CAGE_NAME
            ORDER BY T1.CAGE_SEQ
        """)
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data :
        cage_index = set([a[0:2] for a in data])
        cage_list = []
        for cage in cage_index :
            cage_info = {'cage_seq':cage[0], 'cage_name':cage[1], 'pet': []}
            cage_info['pet'] += [{'pet_type':i[2], 'pet_name': i[3]} for i in data if i[0:2] == cage]
            cage_list.append(cage_info)

        return cage_list
    else :
        return None

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for
from markupsafe import escape
from datetime import timedelta
import random


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

global token
global session
global info


@app.route('/')
def home() :
    return render_template('home.html')

@app.route('/(?P<menu_1>\d+)/(?P<menu_2>\d+)(?=^/P<info>\d+)')
def index2_info() :
    if len(session) == 0 :
        if info == '001777' :
            info = '001000'
    else :
        if menu_1 == 'login' :
            info = "705001"
    return render_template(f'{menu_1}_{menu_2}.html')
    
@app.route('/(?P<menu_1>\d+)(?=^/P<info>\d+)')
def index_info() :
    if len(session) == 0 :
        if info == '001777' :
            info = '001000'
        elif menu_1 == 'logout' :
            info = '000000'
    else :
        if menu_1 == 'login' :
            info = "705001"
        elif menu_1 == 'logout' :
            session.pop('user_id', None)
            session.pop('login_by', None)
            session.pop('manager', None)
            if len(session) == 0 :
                info = '000777'
            else :
                info = '000666'
    return render_template(f'{menu_1}.html')


# @app.route('/(?P<menu>\d+)(?=^/P<info>\d+)')
# def index_info() :
#     if len(session) :
#         global session
#         print(f'로그인 : {len(session)}')
#         return render_template(f'{menu}.html')
#     else :
#         # session 없는데 로그인 성공 떴을 때
#         if info == '001777' : 
#             global info
#             info = '001000'
#         return render_template(f'{menu}.html')
#     return render_template('main.html')

# @app.route('/(?P<menu_1>\d+)/(?P<menu_2>\d+)')
# def index2(menu_1, menu_2) :
#     return render_template(f'/{menu_1}/{menu_2}.html')

# @app.route('/(?P<menu_1>\d+)/(?P<menu_2>\d+)(?=^/P<info>\d+)')
# def index2_info(menu_1, menu_2) :
#     return render_template(f'/{menu_1}/{menu_2}.html')

# @app.route('/<info>')
# def index_info(info) :
#     global session
#     if len(session) :
#         print(f'로그인 : {len(session)}')
#         return render_template('main.html')
#     else :
#         # session 없는데 로그인 성공 떴을 때
#         if info == '001777' : 
#             info = '잘못된 경로'
#         return render_template('main.html', info=info)


# @app.route('/signup')
# def signup() : 
#     global session
#     return render_template('signup.html')

# @app.route('/signup/rule')
# def rule() :
#     return render_template('rule.html')



# @app.route('/login')
# def login() :
#     global session
#     print('일반 로그인창 오픈')
    
#     if 'user_id' in session :
#         info = "705001"
#         print('현재 상태 로그인')
#         return render_template('login.html', info = info)
#     else :
#         return render_template('login.html')

# @app.route('/login?info=<info>')
# def login_info(info) :
#     global session
#     print(info) 
#     return render_template('login.html', info = info)

@app.route('/login/result', methods=['POST'])
def result() :
    global session
    if request.method == 'POST' :
        mlogin = request.form['mlogin']
        mid = request.form['mid']
        mpw = ''
        if mlogin == 'naver' : 
            token = request.form['token']
        elif mlogin == 'diamond' :
            mpw = request.form['mpw']
        #login_(mlogin,mid,mpw)
    return login_ck(mlogin,mid,mpw)
        

# @app.route('/logout')
# def logout() :
#     global session
#     if len(session) :
#         session.pop('user_id', None)
#         session.pop('login_by', None)
#         session.pop('manager', None)
#         if len(session) == 0 :
#             info = '로그아웃 성공'
#         else :
#             info = '로그아웃 실패'
#     else :
#         info = "잘못된 경로"
#     print(session)
#     print(f'로그아웃 : {len(session)}')
#     return redirect(url_for('index_info', info=info))


@app.errorhandler(404)
def page_not_found(e):
    return render_template("404.html"), 404    
    
@app.errorhandler(500)
def page_not_found(e):
    return render_template("500.html"), 500

if __name__ == '__main__' :
    app.run(host = '0.0.0.0', port=5020)

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for

app = Flask(__name__)

@app.route('/login')
def login() :
    return render_template('login.html')

if __name__ == '__main__' :
    app.run(host = '0.0.0.0', port=5020)

In [ ]:
!pip install sweetalert --save

In [ ]:
# 로그인 : 001
# 로그아웃 : 000
# 현재 상태 : 705
# 성공 :777
# 실패 : 666
# 경로 오류 : 000

In [ ]:
import cx_Oracle
import os

def login_ck(mlogin,mid,mpw='') :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기    
    conn = cx_Oracle.connect('hr','1234','localhost:1521/xe')
    cursor = conn.cursor()
    if mlogin == 'diamond' :
        cursor.execute(f"SELECT mid, mlogin, manager FROM member where mlogin='diamond' and mid='{mid}'and mpw='{mpw}'")     
    else :
        cursor.execute(f"SELECT mid, mlogin, manager FROM member where mlogin='{mlogin}' and mid='{mid}'")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    
    if len(data):
        data = data[0]
        session['user_id'] =  data[0]
        session['login_by'] = data[1]
        session['manager'] = data[2]
        return redirect(url_for('index_info', info='001777', user_id = session['user_id']))
    else :
        return redirect(url_for('login_info', info='001666'))

def select_all(mlogin,mid) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('hr','1234','localhost:1521/xe')
    cursor = conn.cursor()
    cursor.execute(f"SELECT mname, mid, mtel, mmail, mlogin, manager FROM member where mlogin='{mlogin}' and mid='{mid}'")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data:
        data = data[0]
        return data
    else :
        return render_template('/404')

def register():
    
    mno = f'm{int(cursor.execute("select count(*) from member").fetchall()[0]) + 1:0<4}'
    cursor.execute(f"insert into member values('m0002', '도도', 'dodo', '1234','01012341234','dodo@gmail.com',0,'22/08/15','22/08/15');")  
  


In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for
from markupsafe import escape
from datetime import timedelta
import random


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

token = ''
name = ''
info = ''
@app.route('/')
def index() :
    return render_template('home.html', session=session)

@app.route('/<info>/<user_id>')
def index_info(info,user_id) :
    if len(session) == 0 :
        # session 없는데 로그인 성공 떴을 때
        if info == '001777' :
            info = '001000'
    return render_template('home.html', info=info, session=session)


@app.route('/signup')
def signup() :
    return render_template('signup.html', session=session)

@app.route('/signup/rule')
def rule() :
    return render_template('rule.html', session=session)


@app.route('/login')
def login() :
    return render_template('login.html', session=session)

@app.route('/login/<info>')
def login_info(info) :
    if 'user_id' in session :
        info = "705001"
        
    return render_template('login.html', info=info, session=session)
    

@app.route('/login/result', methods=['POST'])
def result() :
    if request.method == 'POST' :
        mlogin = request.form['mlogin']
        mid = request.form['mid']
        mpw = ''
        if mlogin == 'naver' : 
            token = request.form['token']
        elif mlogin == 'diamond' :
            mpw = request.form['mpw']
        #login_(mlogin,mid,mpw)
    return login_ck(mlogin,mid,mpw)
        

@app.route('/logout')
def logout() :
    if len(session) :
        session.pop('user_id', None)
        session.pop('login_by', None)
        session.pop('manager', None)
        if len(session) == 0 :
            info = '000777'
        else :
            info = '000666'
    else :
        info = "000000"
    print(session)
    print(f'로그아웃 : {len(session)}')
    return redirect(url_for('index_info', info=info))

@app.route('/diamond/team')
def diamond() :
    return render_template('diamond_team.html', session=session)

@app.route('/diamond/shop')
def shop() :
    return render_template('diamond_shop.html', session=session)

@app.route('/diamond/contact')
def contact() :
    return render_template('diamond_contact.html', session=session)

@app.route('/community/free')
def community() :
    return render_template('cmmond_free.html', session=session)

@app.errorhandler(404)
def page_not_found(e):
    return render_template("404.html"), 404    
    
@app.errorhandler(500)
def page_not_found(e):
    return render_template("500.html"), 500

if __name__ == '__main__' :
    app.run(host = '0.0.0.0', port=5020)

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for
from datetime import timedelta
import random


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

token = ''
name = ''
info = ''
@app.route('/')
def index() :
    return render_template('home.html', session=session)

@app.route('/<info>/<user_id>')
def index_info(info,user_id) :
    if len(session) == 0 :
        # session 없는데 로그인 성공 떴을 때
        if info == '001777' :
            info = '001000'
    return render_template('home.html', info=info, session=session)


@app.route('/signup')
def signup() :
    return render_template('signup.html', session=session)

@app.route('/signup/rule')
def rule() :
    return render_template('rule.html', session=session)


@app.route('/login')
def login() :
    return render_template('login.html', session=session)

@app.route('/login/<info>')
def login_info(info) :
    if 'user_id' in session :
        info = "705001"
        
    return render_template('login.html', info=info, session=session)
    

@app.route('/login/result', methods=['POST'])
def result() :
    if request.method == 'POST' :
        mlogin = request.form['mlogin']
        mid = request.form['mid']
        mpw = ''
        if mlogin == 'naver' : 
            token = request.form['token']
        elif mlogin == 'diamond' :
            mpw = request.form['mpw']
        #login_(mlogin,mid,mpw)
    return login_ck(mlogin,mid,mpw)
        

@app.route('/logout')
def logout() :
    if len(session) :
        session.pop('user_id', None)
        session.pop('login_by', None)
        session.pop('manager', None)
        if len(session) == 0 :
            info = '000777'
        else :
            info = '000666'
    else :
        info = "000000"
    print(session)
    print(f'로그아웃 : {len(session)}')
    return redirect(url_for('index_info', info=info))

@app.route('/diamond/team')
def diamond() :
    return render_template('diamond_team.html', session=session)

@app.route('/diamond/shop')
def shop() :
    return render_template('diamond_shop.html', session=session)

@app.route('/diamond/contact')
def contact() :
    return render_template('diamond_contact.html', session=session)

@app.route('/community/free')
def community() :
    return render_template('cmmond_free.html', session=session)

@app.errorhandler(404)
def page_not_found(e):
    return render_template("404.html"), 404    
    
@app.errorhandler(500)
def page_not_found(e):
    return render_template("500.html"), 500

if __name__ == '__main__' :
    app.run(host = '0.0.0.0', port=5020)